# Исследование надёжности заёмщиков


**Задача исследования**

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Наше исследование будет состоять из 4 важных этапов**:

1 [Знакомство с данными](#1)

2 [Предобработка данных](#2)

3 [Ответы на вопросы:](#3)

-есть ли зависимость между наличием детей и возвратом кредита в срок?

-есть ли зависимость между семейным положением и возвратом кредита в срок?

-есть ли зависимость между уровнем дохода и возвратом кредита в срок?

-как разные цели кредита влияют на его возврат в срок?

4 [Итоговый вывод](#4)

*Ну что, поехали!*


## Знакомство с данными
<a id="1"></a>

In [1]:
#импортируем библиотеку pandas, прочтем файл и выведем на экран середину таблицы из 5 строк для ознакомления  с данными
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
3629,0,-4141.231545,45,среднее,1,женат / замужем,0,M,сотрудник,0,126624.639333,покупка жилья
6220,0,-1711.670801,51,высшее,0,гражданский брак,1,F,компаньон,0,138315.056934,на покупку автомобиля
11414,0,337434.585642,60,среднее,1,женат / замужем,0,F,пенсионер,0,202746.224275,операции со своей недвижимостью
1985,3,-823.339980,33,среднее,1,женат / замужем,0,F,госслужащий,0,57541.895996,строительство жилой недвижимости
5430,2,-3051.244321,36,неоконченное высшее,2,женат / замужем,0,F,компаньон,0,99065.183975,покупка жилья для семьи


In [2]:
#с помощью метода info ознакомимся с общем информацией о данных в таблице
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Что было выявлено при первичном знакомстве с данными? 

Во-первых, хотим отметить, что названия колонок указаны строго в соответствии с синтаксисом `python`. Таким образом, можно не менять название колонок. 
Во- вторых, "бросились в глаза" отрицательные значения там, где их быть не должно, например, в колонке общего трудового стажа в днях. Обязательно включим этот пункт в раздел обработки данных. 

Для целей дальнейшего исследования поясним **содержание столбцов**:
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

Для дальнейшего анализа данных необходимо провести **следующие этапы предобработки данных**: 

- проверить и удалить дубликаты;
- проверить также числовые значения в столбцах на наличие отрицательных значений;
- проверить и заменить пропуски в данных: в колонках трудового стажа и совокупного дохода пропущено по 10 % значений. Необходимо их заменить, так как 10 % - это существенная доля, удаление этих строк может повлиять на итоговый результат;
- поменять вещественный тип данных на целочисленный;
- лемматизировать значения с целями, так как многие цели схожи между собой, например, "приоретение автомобиля" и " на покупку подержанного автомобиля"
- категоризировать данные для целей будущего исследования. 

*Не будем терять ни минуты! Приступим к предобработке данных!*

## Предобработка данных
<a id="2"></a>

### Обработка дубликатов

In [3]:
#для начала применим ко всем столбцам с типом данных object метод str.lower() для того,
#чтобы в дальнейшем были учтены возможные дубликаты, которые мы можем не найти из-за регистра
def strlower(columns): # напишем функцию, которая "пройдет" по необходимым нам столбцам и приведет строки к нижнему регистру
    for column in columns:
        data[column] = data[column].str.lower()
columns = ['education', 'family_status', 'gender', 'income_type', 'purpose'] #составим список названий столбцов с типом данных object
strlower(columns) #применим функцию к столбцам с типом данных object
data.head(10) #выведем первые 10 строк таблицы для проверки 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


In [4]:
#теперь посчитаем количество явных дубликатов в таблице:
data.duplicated().sum()

71

In [5]:
#удалим явные дубликаты (с удалением старых индексов и формированием новых) и проверим их отсутствие, еще раз пересчитав дубликаты
data = data.drop_duplicates().reset_index(drop = True)
data.duplicated().sum()

0

In [6]:
#посчитаем уникальные значения в столбцах с образованием, семейным положением, полом клиента и типом занятости, 
#для того чтобы проверить наличие неявных дубликатов
columns1 = ['education', 'family_status', 'gender', 'income_type']
for column in columns1:
    display(data[column].value_counts())

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

f      14174
m       7279
xna        1
Name: gender, dtype: int64

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

**Вывод**

В рамках проверки дубликатов в исходной таблице был найден 71 дубликат или 0,3% от всех данных, что, на наш взгляд, является вполне допустимым, так как возможны ошибки технологического характера, а также ошибки ввиду человеческого фактора. 

Мы удалили дубликаты методом `drop_duplicates()`, выявленные при помощи метода `duplicated`, и далее проверили уникальные значения в столбцах с образованием, семейным положением, полом клиента и типом занятости. 

В результате выявлено, что в столбце с полом клиента, помимо стандартных значений 'f' и 'м', найдено некорректное значение '`xna`', но так как у нас не стоит задача проанализировать данные по гендерному признаку, мы оставили это значение как есть. 

Так как при проверке мы не обнаружили "неявные" дубликаты, то в данном случае удаления методом `drop_duplicates()` достаточно. 

### Обработка отрицательных значений

In [7]:
#Проверим столбцы с данными о наличии детей, возрасте, идентификаторе уровня образования, идентификаторе семейного положения, 
#наличии/отсутствии задолженности и ежемесячном доходе на наличие отрицательных значений:
columns2 = ['days_employed','children','dob_years', 'education_id','family_status_id', 'debt', 'total_income']
for column in columns2:
    if len(data[data[column]<0]) > 0:
        display(f'В столбце {column}  обнаружены отрицательные значения')
    

'В столбце days_employed  обнаружены отрицательные значения'

'В столбце children  обнаружены отрицательные значения'

Получается, что в "числовых" столбцах отрицательные значения присутствуют в столбцах c данными о детях, а также с общим трудовым стажем 
(что также было выявлено при первом знакомстве с данными). 

In [8]:
#С помощью функции abs() заменим значения столбцов на их модуль:
data['children'] = abs(data['children'])
data['days_employed'] = abs(data['days_employed'])
data.head(10) #проверим, что отрицательные значения "ушли"

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,m,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,f,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

В рамках анализа отрицательных значений было выявлено, что отрицательные значения содержатся в столбцах c данными о детях, а также с общим трудовым стажем.

Предположительно, это связано с особенностями ручного заполнения: Клиент мог поставить тире и указать трудовой стаж или наличие детей:
таким образом, система распознала его как отрицательное значение. 

С помощью функции `abs()` мы заменили все значения столбцов на их модуль, сохранив при этом тип данных. 

### Обработка пропусков

In [9]:
# проверим, какие значения пропущены в столбцах с трудовым стажем и совокупным доходом методом isna():
display(data['days_employed'].isna().sum())
display(data['total_income'].isna().sum())

2103

2103

Пропущено одинаковое количество значений в столбцах с трудовым стажем и совокупным доходом. Можно предположить, 
что значения пропущены, так как это Клиенты банка, у которых отсутствует трудовой стаж и, соответственно, доход. 

Проверим нашу гипотезу:

In [10]:
#выведем первые 5 строк таблицы на экран при условии, что значения столбца с трудовым стажем пропущены и убедимся, что значения совокупного дохода также пропущены
display(data[data['days_employed'].isna()].head())
#посчитаем, количество строк, где отсутствуют значения и трудового стажа, и совокупного дохода
display(len(data[(data['days_employed'].isna()) & (data['total_income'].isna())]))
#проверим, какие в этом случае значения присутствуют в столбце с типом занятости
data[(data['days_employed'].isna()) & (data['total_income'].isna())]['income_type'].value_counts()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,m,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,m,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,не женат / не замужем,4,f,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,f,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,f,пенсионер,1,NaN,сыграть свадьбу


2103

сотрудник          1070
компаньон           501
пенсионер           386
госслужащий         145
предприниматель       1
Name: income_type, dtype: int64

Так как количество строк при условии, что отсутствуют значения и трудового стажа, и совокупного дохода, совпало с количеством пропусков в столбцах с трудовым стажем и совокупным доходом, подсчитанных ранее методом isna()- 2103, делаем вывод, что значения пропущены в одних и тех же строках.

Однако, нельзя сделать вывод, что это люди, которые никогда не работали, так как в столбце с типом занятости указаны такие категории, как: сотрудник, компаньон, пенсионер, госслужащий и предприниматель. 

Здесь становится очевидным, что 10% заполняющих анкеты на кредит по каким-то своим личным причинам не указывают совокупный стаж и доход. 

Заменим пропущенные значения медианными по каждой группе по типу занятости.

In [11]:
#заменим пропущенные значения в столбцах с трудовым стажем и совокупным доходом на 0 методом fillna()
data['days_employed'] = data['days_employed'].fillna(0)
data['total_income'] = data['total_income'].fillna(0)
#Вычислим медиану по каждому типу занятости для дохода и трудового стажа, заменим нулевые значения медианой:
median_income = data.groupby (by = 'income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].replace(0,median_income)
median_employed = data.groupby (by = 'income_type')['days_employed'].transform('median')
data['days_employed'] = data['days_employed'].replace(0,median_employed)
#проверим, как прошла замена, вызвав команду для просмотра середины таблицы(5 строк). Первая же строка ранее была с NaN
data.sample(5)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16649,0,1175.865007,55,среднее,1,не женат / не замужем,4,f,сотрудник,0,82036.513669,на покупку подержанного автомобиля
4169,0,107.155892,50,высшее,0,гражданский брак,1,m,компаньон,0,110604.093367,сыграть свадьбу
12083,1,907.554827,28,неоконченное высшее,2,не женат / не замужем,4,f,сотрудник,0,64212.633829,операции со своей недвижимостью
6543,0,1568.386504,27,среднее,1,гражданский брак,1,m,сотрудник,0,138982.225897,автомобиль
20121,2,1366.901286,42,среднее,1,женат / замужем,0,f,сотрудник,0,133890.827873,покупка недвижимости


In [12]:
#еще раз вызовем метод info(), чтобы убедиться, что пропущенных значений больше нет
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null float64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В рамках обработки пропущенных значений в таблице были выявлены пропуски типа `NaN` в столбцах с трудовым стажем и совокупным доходом.

Было выяснено, что значения пропущены в одних и тех же строках, при этом в этих строках в столбце с типом занятости указаны такие категории, как: сотрудник, компаньон, пенсионер, госслужащий и предприниматель. 

Стало очевидным, что 10% заполняющих анкеты на кредит по каким-то своим личным причинам не указывают совокупный стаж и доход.

Пропущенные значения мы заменили медианными для каждой группы, выделенной по типу занятости, поскольку медианные значения минимализируют влияние выбросов, в отличии, например, от среднего. 

### Замена вещественного типа данных на целочисленный

In [13]:
#проведем замену типа данных в столбцах с трудовым стажем и совокупным доходом (float на int с помощью метода astype()):
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')
#проверим успешность замены типа данных, вызвав метод info():
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Для удобства дальнейшего исследования (выделения категорий) мы заменили тип данных - вещественный на целочисленный.
Использовали метод `astype()`, поскольку данным методом можно привести значения к нужному типу, например, с аргументом `('int')` - в целое число. 

### Лемматизация целей кредита

In [14]:
#с помощью метода join() создадим новую переменную text - строку всех значений столбца с целями кредита
text = ','.join(data['purpose'])
text[0:1000]

'покупка жилья,приобретение автомобиля,покупка жилья,дополнительное образование,сыграть свадьбу,покупка жилья,операции с жильем,образование,на проведение свадьбы,покупка жилья для семьи,покупка недвижимости,покупка коммерческой недвижимости,сыграть свадьбу,приобретение автомобиля,покупка жилой недвижимости,строительство собственной недвижимости,недвижимость,строительство недвижимости,на покупку подержанного автомобиля,на покупку своего автомобиля,недвижимость,приобретение автомобиля,на покупку подержанного автомобиля,сыграть свадьбу,операции с коммерческой недвижимостью,покупка недвижимости,образование,строительство собственной недвижимости,строительство собственной недвижимости,строительство жилой недвижимости,операции с коммерческой недвижимостью,жилье,на проведение свадьбы,покупка жилья,операции со своей недвижимостью,на проведение свадьбы,автомобили,приобретение автомобиля,образование,заняться образованием,покупка недвижимости,сделка с подержанным автомобилем,получение образования,

In [15]:
#импортируем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()
#лемматизируем цели кредита
lemmas = m.lemmatize(text)
#вызовем специальный контейнер Counter из модуля collections
from collections import Counter
#подсчитаем число упоминаний лемматизированных слов в тексте
Counter(lemmas)

Counter({'покупка': 5897,
         ' ': 33570,
         'жилье': 4460,
         ',': 21453,
         'приобретение': 461,
         'автомобиль': 4306,
         'дополнительный': 906,
         'образование': 4013,
         'сыграть': 765,
         'свадьба': 2324,
         'операция': 2604,
         'с': 2918,
         'на': 2222,
         'проведение': 768,
         'для': 1289,
         'семья': 638,
         'недвижимость': 6351,
         'коммерческий': 1311,
         'жилой': 1230,
         'строительство': 1878,
         'собственный': 635,
         'подержать': 853,
         'свой': 2230,
         'со': 627,
         'заниматься': 904,
         'сделка': 941,
         'получение': 1314,
         'высокий': 1374,
         'подержанный': 111,
         'профильный': 436,
         'сдача': 651,
         'ремонт': 607,
         '\n': 1})

Выделим несколько наиболее часто встречающихся целей:
- 'жилье', 'недвижимость': 10811 (объединим в 'недвижимость')
- 'автомобиль': 4306
- 'образование': 4013
- 'свадьба': 2324

In [16]:
#создадим функцию
def lemmatize_purpose(purpose):
    if ('жил'in purpose) or ('недвижимост' in purpose):
        return 'недвижимость'
    if 'автомобил' in purpose:
        return 'автомобиль'
    if 'образован' in purpose:
        return 'образование'
    if 'свадьб' in purpose:
        return 'свадьба'
    return 'другое'
#создадим новый столбец в таблице для лемматизированных целей
data['lemmatize_purpose'] = data['purpose'].apply(lemmatize_purpose)
#проверим, сколько целей не получилось лемматизировать
data['lemmatize_purpose'].value_counts()

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: lemmatize_purpose, dtype: int64

**Вывод**

Для лемматизации целей кредита, в первую очередь, мы создали новую переменную - строку из столбца с целями, для того, чтобы передать ее функции `lemmatize` из библиотеки `pymystem3`.

Далее, лемматизировав полученные значения, подсчитали число упоминаний лемматизированных слов в тексте. 

Затем мы выделили наиболее часто встречающиеся цели кредитования из списка лемматизированных слов и написали функцию, с помощью которой в новый столбец таблицы добавили уже лемматизированные цели. 

Таким образом, у нас получилось **4 основных цели кредитования**:
*недвижимость, автомобиль, образование, свадьба*. 

### Категоризация данных

Для целей будущего исследования данные необходимо категоризировать. 

Нам будет необходимо дать ответы на следующие вопросы:
- есть ли зависимость между наличием детей и возвратом кредита в срок?
- есть ли зависимость между семейным положением и возвратом кредита в срок?
- есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- как разные цели кредита влияют на его возврат в срок?

Таким образом, необходимо выделить категории для столбцов с данными о:

- наличии детей (выделим 3 категории: 'нет детей' , '1-2 ребенка', '3 и более детей');
- семейном положении (здесь категории уже выделены, но необходим словарь, так как с "длинными" названиями работать не так удобно);
- ежемесячном доходе(выделим основные группы: 'средний' , 'ниже среднего', 'высокий'доход, а также отдельную группу с нулевыми значениями - '0').

Для столбца с целями мы уже выделили 4 группы целей в предыдущем пункте. Напомним, это: 'недвижимость', 'автомобиль', 'образование', 'свадьба'.

 *Выделим категории для столбца с данными о наличии детей*

In [17]:
#в первую очередь, проверим, какие значения есть в столбце
data['children'].value_counts()

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Да, 20 детей - такое возможно. Дети могут быть, например, приемными. Но 20 детей в 76 случаях- это уже похоже на ошибку в данных. Скорее всего, здесь лишний ноль, поэтому заменим значение 20 на 2. 

In [18]:
#заменим значения 20 на 2
data.loc[data['children'] == 20, 'children'] = 2
#проверим замену значений:
data['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Данные столбца готовы для категоризации. Выделим 3 категории: 'нет детей' , '1-2 ребенка', '3 и более детей'.

In [19]:
#запишем функцию
def children_group(children):
    if children == 0:
        return 'нет детей'
    if (children == 1) or (children == 2):
        return '1-2 ребенка'
    return '3 и более детей'
#создадим новый столбец с категориями по наличию детей
data['children_group'] = data['children'].apply(children_group)
#проверка
display(data.head(10))
data['children_group'].value_counts()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,children_group
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1-2 ребенка
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,3 и более детей
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,нет детей
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,1-2 ребенка
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей


нет детей          14091
1-2 ребенка         6983
3 и более детей      380
Name: children_group, dtype: int64

*Создадим словарь для столбца с данными о семейном положении*

In [20]:
family_status_dict = data[['family_status', 'family_status_id']].drop_duplicates().reset_index(drop=True)
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,не женат / не замужем,4


In [21]:
#посчитаем количество значений каждой группы толбца с данными о семейном положении методом value.counts()
data['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

 *Выделим категории для столбца с данными о ежемесячном доходе*

In [22]:
#для начала посмотрим максимальное, минимальное и медианное и среднее значение для столбца с данными о ежемесячном доходе
display(data['total_income'].max())
display(data['total_income'].min())
display(data['total_income'].median().astype('int'))
display(data['total_income'].mean().astype('int'))

2265604

20667

140215

164429

In [23]:
#найдем значения ежемесячного дохода для квантилей 0,33 и 0, 66, чтобы статистически разделить столбец на 3 группы
import numpy as np
np.quantile(data['total_income'], q = [0.33, 0.66])

array([118297.29, 169342.94])

Выделим следующие категории:
- 'средний' - значения от 118000 до 169000
- 'ниже среднего' - значения до 118000
- 'высокий' - значения от 169000 и выше

In [24]:
#запишем функцию
def income_group(total_income):
    if total_income < 119000:
        return 'ниже среднего'
    if (total_income >= 119000) & (total_income < 170000):
        return 'средний'
    return 'высокий'
#создадим новый столбец с категориями по доходам
data['income_group'] = data['total_income'].apply(income_group)
#проверка
display(data.head(10))
data['income_group'].value_counts()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmatize_purpose,children_group,income_group
0,1,8437,42,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,недвижимость,1-2 ребенка,высокий
1,1,4024,36,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,автомобиль,1-2 ребенка,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,недвижимость,нет детей,средний
3,3,4124,32,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,образование,3 и более детей,высокий
4,0,340266,53,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,свадьба,нет детей,средний
5,0,926,27,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,недвижимость,нет детей,высокий
6,0,2879,43,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,недвижимость,нет детей,высокий
7,0,152,50,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,образование,нет детей,средний
8,2,6929,35,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,свадьба,1-2 ребенка,ниже среднего
9,0,2188,41,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,недвижимость,нет детей,средний


высокий          7217
ниже среднего    7127
средний          7110
Name: income_group, dtype: int64

**Вывод**

Итак, мы подготовили данные для дальнейшего исследования.

Выделены категории для столбцов с данными о:

- наличии детей (3 категории: 'нет детей' , '1-2 ребенка', '3 и более детей');
- семейном положении (создан словарь со значениями 'женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'не женат / не замужем' и соответствующими им id);
- ежемесячном доходе(выделены группы по доходам: 'средний'(от 119000 до 170000) , 'ниже среднего'(до 119000), 'высокий' (от 119000)).
- целях (выделены 4 группы целей: 'недвижимость', 'автомобиль', 'образование', 'свадьба').

*Можем приступать к исследованию!*

## Ответы на вопросы
<a id="3"></a>

Перед тем, как ответить на вопросы влияния тех или иных факторов на возврат кредита в срок, давайте посмотрим, а какой % просроченной задолженности в целом по данным всей таблицы

In [25]:
#посчитаем значения по столбцу с данными о задолженности с помощью метода value_counts()
data['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

Таким образом, в среднем 8 % заемщиков имеют просроченную задолженность по кредитам. 

- **Есть ли зависимость между наличием детей и возвратом кредита в срок?**

In [26]:
#для проверки наличия зависимости между количеством детей и возвратом кредита в срок сгруппируем данные с помощью 
#метода groupby и agg
data_grouped_children = data.groupby(['children_group', 'debt']).agg({'debt':'count'})
#для расчета процентов добавим столбец с суммарным значением по каждой категории
data_grouped_children['sum'] = [6983,6983,380, 380, 14091, 14091]
#добавим столбец для расчета процентов
data_grouped_children['percent'] = (data_grouped_children['debt']/data_grouped_children['sum']*100).astype('int')
#заменим название столбца 'debt' на 'debt_count', чтобы избежать путаницу в дальнейшем с уровнем индекса
data_grouped_children.set_axis(['debt_count', 'sum', 'percent'], axis='columns', inplace=True)
#выведем таблицу на экран
data_grouped_children

debt_count    sum  percent
children_group  debt                            
1-2 ребенка     0           6336   6983       90
                1            647   6983        9
3 и более детей 0            349    380       91
                1             31    380        8
нет детей       0          13028  14091       92
                1           1063  14091        7

In [27]:
#посчитаем по-другому
data_grouped_children_new = data.pivot_table(index='children',values='debt',aggfunc=['mean','count'])
data_grouped_children_new

,mean,count
,debt,debt
children,,
0,0.075438,14091
1,0.091658,4855
2,0.094925,2128
3,0.081818,330
4,0.097561,41
5,0.000000,9


**Вывод**

В рамках анализа данных по наличию детей и возврату кредита в срок мы рассмотрели 2 ситуации:

Первая: Рассмотрели зависимость по группам заемщиков: без детей, 1-2 ребенка, 3 и более детей. Здесь существенной зависимости не выявлено. 

Во всех 3 категориях: заемщики без детей, с 1-2 детьми, с 3 и более детей, - более чем 90 % заемщиков не имеют просроченной задолженности по возврату кредитов. Чуть лучше ситуация по заемщикам, у которых нет детей, на 2 % хуже- у заемщиков с 1-2 детьми. 
При этом значения близки к среднему по таблице - а это 8  % , поэтому говорить о какой-то явной зависимости здесь мы не можем. 

Для более наглядного представления выведем на экран сводную таблицу с полученными данными (%)- ниже.

Вторая: Рассмотрели зависимость, не группируя заемщиков по количеству детей, то есть отдельно для заемщиков без детей, с 1 ребенком, 
2, 3, 4 и 5 детьми. 

Что показал этот анализ? Заемщики с 5 детьми вообще не имеют просрочек по кредитам, однако таких заемщиков всего 9, в то время как заемщиков без детей 14091, поэтому мы не можем говорить, что эти выводы полностью обоснованы. 

Остальные заемщики по количеству просрочек распределились следующим образом: с 1,2 и 4 детьми заемщики имеют просрочки по задолженности в 9 % случаев, с 3 детьми- в 8% , и заемщики без детей- в 7,5 % случаев, что даже лучше (на 0,5 %) среднего показателя 8 %. 

Скорее всего, это связано с тем, что у клиентом без детей меньше обязательств и непредвиденных трат или непредвиденных больничных, из-за которых также может снижаться доход и возникать ситуация просрочки. 

In [28]:
#выведем на экран сводную таблицу с результатами с помощью метода pivot_table()
data_grouped_children_pivot = data_grouped_children.pivot_table(index='children_group', columns='debt', values='percent')
data_grouped_children_pivot

debt,0,1
children_group,,
1-2 ребенка,90,9
3 и более детей,91,8
нет детей,92,7


- **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [29]:
#для проверки наличия зависимости между семейным положением заемщика и возвратом кредита в срок сгруппируем данные с помощью 
#метода groupby и agg
data_grouped_family_status = data.groupby(['family_status', 'debt']).agg({'debt':'count'})
#для расчета процентов добавим столбец с суммарным значением по каждой категории
data_grouped_family_status['sum'] = [1195,1195, 959, 959, 4151, 4151, 12339,12339, 2810, 2810]
#добавим столбец для расчета процентов
data_grouped_family_status['percent'] = (data_grouped_family_status['debt'] / data_grouped_family_status['sum'] * 100).astype('int')
#заменим название столбца 'debt' на 'debt_count', чтобы избежать путаницу в дальнейшем с уровнем индекса
data_grouped_family_status.set_axis(['debt_count', 'sum', 'percent'], axis='columns', inplace=True)
#выведем таблицу на экран
data_grouped_family_status

debt_count    sum  percent
family_status         debt                            
в разводе             0           1110   1195       92
                      1             85   1195        7
вдовец / вдова        0            896    959       93
                      1             63    959        6
гражданский брак      0           3763   4151       90
                      1            388   4151        9
женат / замужем       0          11408  12339       92
                      1            931  12339        7
не женат / не замужем 0           2536   2810       90
                      1            274   2810        9

**Вывод**

В рамках анализа данных по семейному положению и возврату кредита в срок *зависимость между семейным положением и возвратом кредита в срок* **НЕ выявлена**.

Во всех 5 категориях заемщиков более чем 90 % заемщиков не имеют просроченной задолженности по возврату кредитов. Немного хуже, чем у остальных категорий, ситуаций у заемщиков, состоящих в гражданском браке или неженатых/незамужных: просроченную задолженность допускает почти 10  % таких заемщиков. Однако эти данные не сильно различаются со средним значением просроченной задолженности по всей таблице (10% против 8%). Вдовцы и вдовы согласно анализа допускают меньше всего просрочек по кредиту (6%).

Для более наглядного представления выведем на экран сводную таблицу с полученными данными (%).

In [30]:
#выведем на экран сводную таблицу с результатами с помощью метода pivot_table()
data_grouped_family_status_pivot = data_grouped_family_status.pivot_table(index='family_status', columns='debt', values='percent')
data_grouped_family_status_pivot

debt,0,1
family_status,,
в разводе,92,7
вдовец / вдова,93,6
гражданский брак,90,9
женат / замужем,92,7
не женат / не замужем,90,9


- **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [31]:
#для проверки наличия зависимости между уровнем дохода заемщика и возвратом кредита в срок сгруппируем данные с помощью 
#метода groupby и agg
data_grouped_income_group = data.groupby (['income_group', 'debt']).agg({'debt': 'count'})
#для расчета процентов добавим столбец с суммарным значением по каждой категории
data_grouped_income_group['sum'] = [7217, 7217, 7127, 7127, 7110, 7110]
#добавим столбец для расчета процентов
data_grouped_income_group['percent'] = (data_grouped_income_group['debt'] / data_grouped_income_group['sum'] * 100).astype('int')
#заменим название столбца 'debt' на 'debt_count', чтобы избежать путаницу в дальнейшем с уровнем индекса
data_grouped_income_group.set_axis(['debt_count', 'sum', 'percent'], axis='columns', inplace=True)
data_grouped_income_group

debt_count   sum  percent
income_group  debt                           
высокий       0           6670  7217       92
              1            547  7217        7
ниже среднего 0           6547  7127       91
              1            580  7127        8
средний       0           6496  7110       91
              1            614  7110        8

**Вывод**

Как и в предыдущих случаях, *анализ данных по уровню дохода* **НЕ выявил** каких-либо серьезных отклонений от нормы в плане возврата кредита в срок. По всем категориям клиентов в зависимости от уровня дохода, процент заемщиков, которые допускали задолженность по кредиту - 7-8 % (в среднем по таблице 8 %).

Для более наглядного представления выведем на экран сводную таблицу с полученными данными (%).

In [32]:
#выведем на экран сводную таблицу с результатами с помощью метода pivot_table()
data_grouped_income_group_pivot = data_grouped_income_group.pivot_table(index='income_group', columns='debt', values='percent')
data_grouped_income_group_pivot

debt,0,1
income_group,,
высокий,92,7
ниже среднего,91,8
средний,91,8


- **Как разные цели кредита влияют на его возврат в срок?**

In [33]:
#для проверки наличия зависимости между целями кредита и возвратом кредита в срок сгруппируем данные с помощью 
#метода groupby и agg
data_grouped_lemmatize_purpose = data.groupby(['lemmatize_purpose', 'debt']).agg({'debt':'count'})
#для расчета процентов добавим столбец с суммарным значением по каждой категории
data_grouped_lemmatize_purpose['sum'] = [4306, 4306, 10811, 10811, 4013, 4013, 2324, 2324]
#добавим столбец для расчета процентов
data_grouped_lemmatize_purpose['percent'] = (data_grouped_lemmatize_purpose['debt'] / data_grouped_lemmatize_purpose['sum'] * 100).astype('int')
#заменим название столбца 'debt' на 'debt_count', чтобы избежать путаницу в дальнейшем с уровнем индекса
data_grouped_lemmatize_purpose.set_axis(['debt_count', 'sum', 'percent'], axis='columns', inplace=True)#выведем таблицу на экран
#выведем таблицу на экран
data_grouped_lemmatize_purpose

debt_count    sum  percent
lemmatize_purpose debt                            
автомобиль        0           3903   4306       90
                  1            403   4306        9
недвижимость      0          10029  10811       92
                  1            782  10811        7
образование       0           3643   4013       90
                  1            370   4013        9
свадьба           0           2138   2324       91
                  1            186   2324        8

**Вывод**

*Анализ данных по целям кредита* в разрезе возврата/невозврата кредита в срок также существенных отклонений **НЕ выявил**.

Чуть больше среднего значения(8%) процент просроченных задолженностей по кредитам на автомобиль и образование - 10%. 
Чуть меньше - 7 % - по кредитам на недвижимость. 

Для более наглядного представления выведем на экран сводную таблицу с полученными данными (%).

In [34]:
#выведем на экран сводную таблицу с результатами с помощью метода pivot_table()
data_grouped_lemmatize_purpose_pivot = data_grouped_lemmatize_purpose.pivot_table(index='lemmatize_purpose', columns='debt', values='percent')
data_grouped_lemmatize_purpose_pivot

debt,0,1
lemmatize_purpose,,
автомобиль,90,9
недвижимость,92,7
образование,90,9
свадьба,91,8


## Итоговый вывод
<a id="4"></a>

Итак, мы провели анализ данных из банковского файла со статистикой о платёжеспособности клиентов. 

В первую очередь, мы ознакомились с общими данными из файла, удалили дубликаты, обработали некорректные отрицательные значения, заполнили пропущенные значения, заменили тип данных там, где это необходимо. 

Далее, для того, чтобы ответить на поставленные в задаче исследования вопросы, 
(давайте пройдемся по ним еще раз:
- есть ли зависимость между наличием детей и возвратом кредита в срок?
- есть ли зависимость между семейным положением и возвратом кредита в срок?
- есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- как разные цели кредита влияют на его возврат в срок?)

были категоризированы данные по наличию детей, семейному положению, уровню дохода и целям кредита. 

Были выделены следующие категории для данных о:

- наличии детей (3 категории: 'нет детей' , '1-2 ребенка', '3 и более детей');
- семейном положении (категории: 'женат / замужем', 'гражданский брак', 'вдовец / вдова', 'в разводе', 'не женат / не замужем');
- ежемесячном доходе(выделены группы по доходам: 'средний', 'ниже среднего', 'высокий').
- целях (выделены 4 группы целей: 'недвижимость', 'автомобиль', 'образование', 'свадьба').

Если смотреть данные по всей таблице, без разбивки на категории, в среднем 8 % заемщиков имели просроченную задолженность по кредитам.

В рамках анализа зависимости между возвратом кредита в срок и наличием детей/ семейным положением/уровнем дохода/целями кредиты критические отклонения, которые бы давали нам возможность дать рекомендации для будущей модели кредитного скоринга об однозначном отказе или, наоборот, одобрении кредитной заявки по определенной категории заемщика/цели кредита, не выявлены. 

Значения невозврата кредита в срок по разным категориям колеблются от 6 до 10 %, то есть явных отклонений не выявлено. 

В то же время банку следует обратить особое внимание на категории заемщиков, где процент невозврата доходит до 10 % (что на 25% выше средних 8%) и применять дополнительные контрольные меры для принятия решения о выдаче кредита. 
Например, выборочный "ручной" контроль таких заявок. Или дополнительные вопросы анкеты, по которым будет принято более "взвешенное" решение (к примеру, вопросы о здоровье, о возможности потери работы, о финансовой подушке и т.д.)

Давайте перечислим категории, в которых согласно исследования процент задолженностей по возврату кредита достигает порядка 10 %:

- по наличию детей - это заемщики с 1-2 детьми и 4 детьми;
- по семейному положению - заемщики, которые состоят в гражданском браке или не женаты/не замужем;
- по целям - заемщики, которые берут кредит на автомобиль или образование. 

Ну и конечно же, модель кредитного скоринга - это сложная система, и  для ее построения необходимо собрать максимум данных и провести большое глобальное исследование, начало которому уже положено в этом исследовании. 
